### **5.1 Load packages**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
# ignore warnings
import warnings
warnings.filterwarnings("ignore")


### **5.2 Read dataset**

In [2]:
# load dataset
df = pd.read_csv('./bike.csv')